<a href="https://colab.research.google.com/github/eduardodut/Mineracao_dados_textos_web/blob/master/projeto01_equipe01_entrega18052020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b> EQUIPE: </b>
  - Eduardo Façanha
  - Giovanni Brígido
  - Maurício Brito

<b> ATIVIDADE 01 </b> - Pré-processamento dos textos (Prazo: 11/05/2020 - 30%)

- Tokenização
- Lematização
- POS Tagging
- Normalização (hashtags, menções, emojis e símbolos especiais)
- NER (entidades nomeadas)
- Remoção stop-words

<b> ATIVIDADE 02 </b> - Representação Semântica (Prazo: 22/06/2020 - 30%)

- Uso de bases de conhecimento externas
- Identificação de tópicos
- Representação vetorial das palavras e textos

<b> ATIVIDADE 03 </b> - Analise da Linguagem Ofensiva - Subtarefas A e B (Prazo: 27/07/2020 - 40%)

- Resultado da subtarefa A para um conjunto de teste a ser fornecido
- Resultado da subtarefa B para um conjunto de teste a ser fornecido


<b> Carregamento do arquivo de dados e transformação em DataFrame </b>

É realizado o download do arquivo e instanciado um DataFrame com os dados. A variável do DataFrame é chamada 'tweets'

In [29]:
import pandas as pd
#download o arquivo localizado no reposítório do projeto
!curl --remote-name \
    -H 'Accept: application/vnd.github.v3.raw' \
    --location https://raw.githubusercontent.com/eduardodut/Mineracao_dados_textos_web/master/datasets/olid-training-v1.0.tsv

#leitura para objeto dataframe
tweets = pd.read_csv('/content/olid-training-v1.0.tsv', sep='\t',encoding= 'utf-8')

#conversão da coluna 'id' de inteiro para string
tweets['id'] = tweets['id'].astype('str')

#visualização dos primeiros registros

tweets = tweets[['subtask_c','subtask_b','subtask_a','id','tweet']]
tweets.head(20)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1915k  100 1915k    0     0  6200k      0 --:--:-- --:--:-- --:--:-- 6180k


,subtask_c,subtask_b,subtask_a,id,tweet
0,NaN,UNT,OFF,86426,@USER She should ask a few native Americans wh...
1,IND,TIN,OFF,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...
2,NaN,NaN,NOT,16820,Amazon is investigating Chinese employees who ...
3,NaN,UNT,OFF,62688,"@USER Someone should'veTaken"" this piece of sh..."
4,NaN,NaN,NOT,43605,@USER @USER Obama wanted liberals &amp; illega...
5,OTH,TIN,OFF,97670,@USER Liberals are all Kookoo !!!
6,NaN,UNT,OFF,77444,@USER @USER Oh noes! Tough shit.
7,GRP,TIN,OFF,52415,@USER was literally just talking about this lo...
8,NaN,NaN,NOT,45157,@USER Buy more icecream!!!
9,IND,TIN,OFF,13384,@USER Canada doesn’t need another CUCK! We alr...


In [30]:
#verificação e remoção de duplicatas
if tweets.duplicated(['tweet']).sum()>0:
  tweets.drop_duplicates(subset='tweet', keep='first', inplace=True)

print('VALORES DUPLICADOS: ',tweets.duplicated(['tweet']).sum())

VALORES DUPLICADOS:  0


<b> Tratamento inicial do texto </b>

Converte o texto de cada tweet, separadamente, em minúsculo e remove espaços e tabulações extras. O resultado é guardado no DataFrame tweets em uma nova coluna.

In [62]:
from nltk.tokenize import TweetTokenizer, sent_tokenize
import re
import string
from nltk.corpus import stopwords as sw

def tratamento_texto(tweet):
  
  tweet = tweet.lower()
  tweet = tweet.strip()
  
  #remove as menções a usuários de cada tweet
  tweet = re.sub(r'@user', '', tweet, flags=re.MULTILINE)
  #remove as palavras url
  tweet = re.sub(r'url', '', tweet, flags=re.MULTILINE)
  #remove as quebras de linha
  tweet = re.sub(r'\n', '', tweet)
  #substitui tabulações por um espaço em branco
  tweet = re.sub(r'\t', ' ', tweet)
  #substitui um ou mais espaços em branco por um espaço
  tweet= re.sub(r'\s+', ' ', tweet, flags=re.I)
  #&amp;
  #remove aspas e apóstofres
  # tweet = re.sub('[\'"‘’“”…]', '', tweet)
  return tweet

#cria uma nova coluna no dataframe 'tweets' com cada tweet tokenizado
tweets['tweet_tratado'] = tweets['tweet'].apply(tratamento_texto)
tweets[tweets.columns[::-1]].head()

,tweet_NER,tweet_POS_tagged,tweet_tokenizado,tweet_em_sentencas,tweet_tratado,tweet,id,subtask_a,subtask_b,subtask_c
0,"[(she, PRP), (should, MD), (ask, VB), (a, DT),...","[(she, PRP), (should, MD), (ask, VB), (a, DT),...","[she, should, ask, a, few, native, americans, ...",[she should ask a few native americans what th...,she should ask a few native americans what th...,@USER She should ask a few native Americans wh...,86426,OFF,UNT,NaN
1,"[(go, VB), (home, NN), (youre, NN), (drunk, NN...","[(go, VB), (home, NN), (youre, NN), (drunk, NN...","[go, home, youre, drunk, !, !, !, #maga, #trum...","[go home youre drunk!!!, #maga #trump2020 👊🇺🇸👊]",go home you’re drunk!!! #maga #trump2020 👊🇺🇸👊,@USER @USER Go home you’re drunk!!! @USER #MAG...,90194,OFF,TIN,IND
2,"[(amazon, NN), (is, VBZ), (investigating, VBG)...","[(amazon, NN), (is, VBZ), (investigating, VBG)...","[amazon, is, investigating, chinese, employees...",[amazon is investigating chinese employees who...,amazon is investigating chinese employees who ...,Amazon is investigating Chinese employees who ...,16820,NOT,NaN,NaN
3,"[(someone, NN), (shouldvetaken, VBD), (this, D...","[(someone, NN), (shouldvetaken, VBD), (this, D...","[someone, shouldvetaken, this, piece, of, shit...",[someone shouldvetaken this piece of shit to a...,"someone should'vetaken"" this piece of shit to...","@USER Someone should'veTaken"" this piece of sh...",62688,OFF,UNT,NaN
4,"[(obama, RB), (wanted, VBD), (liberals, NNS), ...","[(obama, RB), (wanted, VBD), (liberals, NNS), ...","[obama, wanted, liberals, &, illegals, to, mov...",[obama wanted liberals &amp; illegals to move ...,obama wanted liberals &amp; illegals to move ...,@USER @USER Obama wanted liberals &amp; illega...,43605,NOT,NaN,NaN


<b> Separação em sentenças </b>

Separa cada tweet em sentenças e os coloca no DataFrame, em uma nova coluna

In [63]:
import nltk
from contextlib import redirect_stdout
import os



with redirect_stdout(open(os.devnull, "w")):
  nltk.download("stopwords") 
  nltk.download('punkt')

def separa_sentencas(tweet):
  
  lista_sentencas = sent_tokenize(tweet)
  # lista_setencas.str.strip()
  nova_lista = []
  for sent in lista_sentencas:
    nova_lista.append(sent.strip())

  return nova_lista #retorna lista de sentenças com a função .strip() aplicada
tweets['tweet_em_sentencas'] = tweets['tweet_tratado'].apply(separa_sentencas)
tweets[tweets.columns[::-1]].head()

,tweet_NER,tweet_POS_tagged,tweet_tokenizado,tweet_em_sentencas,tweet_tratado,tweet,id,subtask_a,subtask_b,subtask_c
0,"[(she, PRP), (should, MD), (ask, VB), (a, DT),...","[(she, PRP), (should, MD), (ask, VB), (a, DT),...","[she, should, ask, a, few, native, americans, ...",[she should ask a few native americans what th...,she should ask a few native americans what th...,@USER She should ask a few native Americans wh...,86426,OFF,UNT,NaN
1,"[(go, VB), (home, NN), (youre, NN), (drunk, NN...","[(go, VB), (home, NN), (youre, NN), (drunk, NN...","[go, home, youre, drunk, !, !, !, #maga, #trum...","[go home you’re drunk!!!, #maga #trump2020 👊🇺🇸👊]",go home you’re drunk!!! #maga #trump2020 👊🇺🇸👊,@USER @USER Go home you’re drunk!!! @USER #MAG...,90194,OFF,TIN,IND
2,"[(amazon, NN), (is, VBZ), (investigating, VBG)...","[(amazon, NN), (is, VBZ), (investigating, VBG)...","[amazon, is, investigating, chinese, employees...",[amazon is investigating chinese employees who...,amazon is investigating chinese employees who ...,Amazon is investigating Chinese employees who ...,16820,NOT,NaN,NaN
3,"[(someone, NN), (shouldvetaken, VBD), (this, D...","[(someone, NN), (shouldvetaken, VBD), (this, D...","[someone, shouldvetaken, this, piece, of, shit...","[someone should'vetaken"" this piece of shit to...","someone should'vetaken"" this piece of shit to...","@USER Someone should'veTaken"" this piece of sh...",62688,OFF,UNT,NaN
4,"[(obama, RB), (wanted, VBD), (liberals, NNS), ...","[(obama, RB), (wanted, VBD), (liberals, NNS), ...","[obama, wanted, liberals, &, illegals, to, mov...",[obama wanted liberals &amp; illegals to move ...,obama wanted liberals &amp; illegals to move ...,@USER @USER Obama wanted liberals &amp; illega...,43605,NOT,NaN,NaN


<b> Tokenização </b>

Reúne as sentenças em uma string única e realiza a tokenização do tweet. O resultado pode ser observado em uma nova coluna.

In [64]:
def tokeniza_sentenca(lista_sentencas):
  tokenizer = TweetTokenizer()
  #união das sentenças
  sentencas_unidas = " ".join(w for w in lista_sentencas)
  #tokenização das sentenças unidas
  tokens = tokenizer.tokenize(sentencas_unidas)

  return tokens

tweets['tweet_tokenizado'] = tweets['tweet_em_sentencas'].apply(tokeniza_sentenca)
tweets[tweets.columns[::-1]].head()

,tweet_NER,tweet_POS_tagged,tweet_tokenizado,tweet_em_sentencas,tweet_tratado,tweet,id,subtask_a,subtask_b,subtask_c
0,"[(she, PRP), (should, MD), (ask, VB), (a, DT),...","[(she, PRP), (should, MD), (ask, VB), (a, DT),...","[she, should, ask, a, few, native, americans, ...",[she should ask a few native americans what th...,she should ask a few native americans what th...,@USER She should ask a few native Americans wh...,86426,OFF,UNT,NaN
1,"[(go, VB), (home, NN), (youre, NN), (drunk, NN...","[(go, VB), (home, NN), (youre, NN), (drunk, NN...","[go, home, you, ’, re, drunk, !, !, !, #maga, ...","[go home you’re drunk!!!, #maga #trump2020 👊🇺🇸👊]",go home you’re drunk!!! #maga #trump2020 👊🇺🇸👊,@USER @USER Go home you’re drunk!!! @USER #MAG...,90194,OFF,TIN,IND
2,"[(amazon, NN), (is, VBZ), (investigating, VBG)...","[(amazon, NN), (is, VBZ), (investigating, VBG)...","[amazon, is, investigating, chinese, employees...",[amazon is investigating chinese employees who...,amazon is investigating chinese employees who ...,Amazon is investigating Chinese employees who ...,16820,NOT,NaN,NaN
3,"[(someone, NN), (shouldvetaken, VBD), (this, D...","[(someone, NN), (shouldvetaken, VBD), (this, D...","[someone, should'vetaken, "", this, piece, of, ...","[someone should'vetaken"" this piece of shit to...","someone should'vetaken"" this piece of shit to...","@USER Someone should'veTaken"" this piece of sh...",62688,OFF,UNT,NaN
4,"[(obama, RB), (wanted, VBD), (liberals, NNS), ...","[(obama, RB), (wanted, VBD), (liberals, NNS), ...","[obama, wanted, liberals, &, illegals, to, mov...",[obama wanted liberals &amp; illegals to move ...,obama wanted liberals &amp; illegals to move ...,@USER @USER Obama wanted liberals &amp; illega...,43605,NOT,NaN,NaN


<b> POS Tagger </b>

Realiza a part of speech tagging do texto de cada token

In [65]:
from contextlib import redirect_stdout
import os

with redirect_stdout(open(os.devnull, "w")):
    nltk.download('averaged_perceptron_tagger')

tweets['tweet_POS_tagged'] = tweets['tweet_tokenizado'].apply(nltk.pos_tag)
tweets[tweets.columns[::-1]].head()

,tweet_NER,tweet_POS_tagged,tweet_tokenizado,tweet_em_sentencas,tweet_tratado,tweet,id,subtask_a,subtask_b,subtask_c
0,"[(she, PRP), (should, MD), (ask, VB), (a, DT),...","[(she, PRP), (should, MD), (ask, VB), (a, DT),...","[she, should, ask, a, few, native, americans, ...",[she should ask a few native americans what th...,she should ask a few native americans what th...,@USER She should ask a few native Americans wh...,86426,OFF,UNT,NaN
1,"[(go, VB), (home, NN), (youre, NN), (drunk, NN...","[(go, VB), (home, NN), (you, PRP), (’, VBP), (...","[go, home, you, ’, re, drunk, !, !, !, #maga, ...","[go home you’re drunk!!!, #maga #trump2020 👊🇺🇸👊]",go home you’re drunk!!! #maga #trump2020 👊🇺🇸👊,@USER @USER Go home you’re drunk!!! @USER #MAG...,90194,OFF,TIN,IND
2,"[(amazon, NN), (is, VBZ), (investigating, VBG)...","[(amazon, NN), (is, VBZ), (investigating, VBG)...","[amazon, is, investigating, chinese, employees...",[amazon is investigating chinese employees who...,amazon is investigating chinese employees who ...,Amazon is investigating Chinese employees who ...,16820,NOT,NaN,NaN
3,"[(someone, NN), (shouldvetaken, VBD), (this, D...","[(someone, NN), (should'vetaken, VBD), ("", PDT...","[someone, should'vetaken, "", this, piece, of, ...","[someone should'vetaken"" this piece of shit to...","someone should'vetaken"" this piece of shit to...","@USER Someone should'veTaken"" this piece of sh...",62688,OFF,UNT,NaN
4,"[(obama, RB), (wanted, VBD), (liberals, NNS), ...","[(obama, RB), (wanted, VBD), (liberals, NNS), ...","[obama, wanted, liberals, &, illegals, to, mov...",[obama wanted liberals &amp; illegals to move ...,obama wanted liberals &amp; illegals to move ...,@USER @USER Obama wanted liberals &amp; illega...,43605,NOT,NaN,NaN


<b> NER </b>

Realiza a reconhecimento de entidades, NER, a partir da coluna com os tokens taggeados do tweet: ['tweet_POS_tagged']

In [66]:
from nltk.tag import pos_tag
from nltk.chunk import conlltags2tree, tree2conlltags
from pprint import pprint
from nltk.chunk.regexp import ChunkString, ChunkRule, ChinkRule 
from nltk.tree import Tree 
from contextlib import redirect_stdout
import os

with redirect_stdout(open(os.devnull, "w")):
    nltk.download('maxent_ne_chunker')
    nltk.download('words')

tweets['tweet_NER'] = tweets['tweet_POS_tagged'].apply(nltk.ne_chunk)
tweets[tweets.columns[::-1]].head()

,tweet_NER,tweet_POS_tagged,tweet_tokenizado,tweet_em_sentencas,tweet_tratado,tweet,id,subtask_a,subtask_b,subtask_c
0,"[(she, PRP), (should, MD), (ask, VB), (a, DT),...","[(she, PRP), (should, MD), (ask, VB), (a, DT),...","[she, should, ask, a, few, native, americans, ...",[she should ask a few native americans what th...,she should ask a few native americans what th...,@USER She should ask a few native Americans wh...,86426,OFF,UNT,NaN
1,"[(go, VB), (home, NN), (you, PRP), (’, VBP), (...","[(go, VB), (home, NN), (you, PRP), (’, VBP), (...","[go, home, you, ’, re, drunk, !, !, !, #maga, ...","[go home you’re drunk!!!, #maga #trump2020 👊🇺🇸👊]",go home you’re drunk!!! #maga #trump2020 👊🇺🇸👊,@USER @USER Go home you’re drunk!!! @USER #MAG...,90194,OFF,TIN,IND
2,"[(amazon, NN), (is, VBZ), (investigating, VBG)...","[(amazon, NN), (is, VBZ), (investigating, VBG)...","[amazon, is, investigating, chinese, employees...",[amazon is investigating chinese employees who...,amazon is investigating chinese employees who ...,Amazon is investigating Chinese employees who ...,16820,NOT,NaN,NaN
3,"[(someone, NN), (should'vetaken, VBD), ("", PDT...","[(someone, NN), (should'vetaken, VBD), ("", PDT...","[someone, should'vetaken, "", this, piece, of, ...","[someone should'vetaken"" this piece of shit to...","someone should'vetaken"" this piece of shit to...","@USER Someone should'veTaken"" this piece of sh...",62688,OFF,UNT,NaN
4,"[(obama, RB), (wanted, VBD), (liberals, NNS), ...","[(obama, RB), (wanted, VBD), (liberals, NNS), ...","[obama, wanted, liberals, &, illegals, to, mov...",[obama wanted liberals &amp; illegals to move ...,obama wanted liberals &amp; illegals to move ...,@USER @USER Obama wanted liberals &amp; illega...,43605,NOT,NaN,NaN


<b> Remoção de stop words </b>

Remove da lista de tokens de cada tweet as stop words da língua inglesa e pontuações.
São retiradas as stop words das colunas ['tweet_ner'] e ['tweet_tokenizado']

In [75]:
from contextlib import redirect_stdout
import os
# import string library function  
from string import punctuation
    


def remove_stop_words(lista_tokens):
  '''Função de remoção de stop word que recebe lista de tokens e devolve
  lista de tokens
  '''
  with redirect_stdout(open(os.devnull, "w")):
    nltk.download("stopwords") 
    nltk.download('punkt')
  
  stopwords = sw.words('english')
  stop_words = set(stopwords + list(punctuation ))

  tokens = [w for w in lista_tokens if not w in stop_words]

  return tokens

def remove_stop_words_tuplas(lista_tuplas):
  '''Função de remoção de stop word que recebe lista de tuplas de token e tag e devolve
  lista de tuplas de token e tag
  '''
  with redirect_stdout(open(os.devnull, "w")):
    nltk.download("stopwords") 
    nltk.download('punkt')
  
  stopwords = sw.words('english')
  stop_words = set(stopwords + list(punctuation ))

  tuplas = [w for w in lista_tuplas if not w[0] in stop_words]

  return tuplas




tweets['tokens_sem_stopwords'] = tweets['tweet_tokenizado'].apply(remove_stop_words)
tweets['NER_sem_stopwords'] = tweets['tweet_NER'].apply(remove_stop_words_tuplas)
tweets[tweets.columns[::-1]].head()

,NER_sem_stopwords,tokens_sem_stopwords,tweet_NER,tweet_POS_tagged,tweet_tokenizado,tweet_em_sentencas,tweet_tratado,tweet,id,subtask_a,subtask_b,subtask_c
0,"[(ask, VB), (native, JJ), (americans, NNS), (t...","[ask, native, americans, take]","[(she, PRP), (should, MD), (ask, VB), (a, DT),...","[(she, PRP), (should, MD), (ask, VB), (a, DT),...","[she, should, ask, a, few, native, americans, ...",[she should ask a few native americans what th...,she should ask a few native americans what th...,@USER She should ask a few native Americans wh...,86426,OFF,UNT,NaN
1,"[(go, VB), (home, NN), (’, VBP), (drunk, NN), ...","[go, home, ’, drunk, #maga, #trump2020, 👊, 🇺, ...","[(go, VB), (home, NN), (you, PRP), (’, VBP), (...","[(go, VB), (home, NN), (you, PRP), (’, VBP), (...","[go, home, you, ’, re, drunk, !, !, !, #maga, ...","[go home you’re drunk!!!, #maga #trump2020 👊🇺🇸👊]",go home you’re drunk!!! #maga #trump2020 👊🇺🇸👊,@USER @USER Go home you’re drunk!!! @USER #MAG...,90194,OFF,TIN,IND
2,"[(amazon, NN), (investigating, VBG), (chinese,...","[amazon, investigating, chinese, employees, se...","[(amazon, NN), (is, VBZ), (investigating, VBG)...","[(amazon, NN), (is, VBZ), (investigating, VBG)...","[amazon, is, investigating, chinese, employees...",[amazon is investigating chinese employees who...,amazon is investigating chinese employees who ...,Amazon is investigating Chinese employees who ...,16820,NOT,NaN,NaN
3,"[(someone, NN), (should'vetaken, VBD), (piece,...","[someone, should'vetaken, piece, shit, volcano...","[(someone, NN), (should'vetaken, VBD), ("", PDT...","[(someone, NN), (should'vetaken, VBD), ("", PDT...","[someone, should'vetaken, "", this, piece, of, ...","[someone should'vetaken"" this piece of shit to...","someone should'vetaken"" this piece of shit to...","@USER Someone should'veTaken"" this piece of sh...",62688,OFF,UNT,NaN
4,"[(obama, RB), (wanted, VBD), (liberals, NNS), ...","[obama, wanted, liberals, illegals, move, red,...","[(obama, RB), (wanted, VBD), (liberals, NNS), ...","[(obama, RB), (wanted, VBD), (liberals, NNS), ...","[obama, wanted, liberals, &, illegals, to, mov...",[obama wanted liberals &amp; illegals to move ...,obama wanted liberals &amp; illegals to move ...,@USER @USER Obama wanted liberals &amp; illega...,43605,NOT,NaN,NaN


<b> Fim da atividade 01 </b>

Tem-se como principais entregas as colunas tweets['tokens_sem_stopwords'] e tweets['NER_sem_stopwords'] do dataset tweets.

In [78]:
tweets[['NER_sem_stopwords','tokens_sem_stopwords']].head()

,NER_sem_stopwords,tokens_sem_stopwords
0,"[(ask, VB), (native, JJ), (americans, NNS), (t...","[ask, native, americans, take]"
1,"[(go, VB), (home, NN), (’, VBP), (drunk, NN), ...","[go, home, ’, drunk, #maga, #trump2020, 👊, 🇺, ..."
2,"[(amazon, NN), (investigating, VBG), (chinese,...","[amazon, investigating, chinese, employees, se..."
3,"[(someone, NN), (should'vetaken, VBD), (piece,...","[someone, should'vetaken, piece, shit, volcano..."
4,"[(obama, RB), (wanted, VBD), (liberals, NNS), ...","[obama, wanted, liberals, illegals, move, red,..."
